Load environment variables from a file `.env` in the current directory.
You can take a look at the `.env.template` file to see how it should look like.

In [60]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.getenv("AZURE_OPENAI_ENDPOINT"))
print(os.getenv("AZURE_OPENAI_API_KEY"))
print(os.getenv("OPENAI_API_VERSION"))

https://crgar-openai-openai.openai.azure.com/
9e109b9266b0496d990f194144a4fad4
2023-10-01-preview


In [61]:
# Create a prompt template
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="podcast player"))

What is a good name for a company that makes podcast player?


In [62]:
# Call the LLM and run the prompt
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain

llm = AzureChatOpenAI(
    #azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    #api_version=os.getenv("OPENAI_API_VERSION"),
    #model_name=os.getenv("OPENAI_MODEL_NAME"),
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
    temperature=0.9) #temperature dictates how whacky the output should be
llmchain = LLMChain(llm=llm, prompt=prompt)
llmchain.run("podcast player")

'PodcastPro.'

In [63]:
# Trying a stock chain: PALChain. This requires langchain-experimental package to be installed
from langchain_experimental.pal_chain import PALChain
palchain = PALChain.from_math_prompt(llm=llm, verbose=True)
palchain.run("If my age is half of my dad's age and he is going to be 60 next year, what is my current age?")



> Entering new PALChain chain...
def solution():
    """If my age is half of my dad's age and he is going to be 60 next year, what is my current age?"""
    dad_age_next_year = 60
    my_age = dad_age_next_year // 2
    result = my_age
    return result


In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools
#from langchain_experimental.agents.agent_toolkits import pal_math   

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
    temperature=0)

tools = load_tools(["pal-math"], llm=llm)

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

ImportError: cannot import name 'pal_math' from 'langchain_experimental.agents.agent_toolkits' (c:\gitrepos\github\crgarcia12\langchain-day1\.venv\Lib\site-packages\langchain_experimental\agents\agent_toolkits\__init__.py)